In [ ]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta
from sqlalchemy import create_engine

# Configuração do Faker para o idioma português do Brasil
fake = Faker('pt_BR')

# Número de registros para cada dataframe
num_clientes = 1000
num_transacoes = 5000
num_emprestimos = 500
num_pagamentos = 2000  # Este número será dinâmico agora

# --- Geração do DataFrame de Clientes ---
clientes_data = []
fake_unique_cpf = Faker('pt_BR')
for _ in range(num_clientes):
    cliente_id = fake.unique.random_number(digits=8)
    nome = fake.name()
    cpf = fake_unique_cpf.unique.cpf()
    data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=70)
    sexo = random.choice(['Masculino', 'Feminino'])
    endereco = fake.address()
    email = fake.email()
    telefone = fake.phone_number()
    clientes_data.append({
        'cliente_id': cliente_id,
        'nome': nome,
        'cpf': cpf,
        'data_nascimento': data_nascimento,
        'sexo': sexo,
        'endereco': endereco,
        'email': email,
        'telefone': telefone
    })
df_clientes = pd.DataFrame(clientes_data)

# --- Geração do DataFrame de Transações ---
transacoes_data = []
tipos_transacao = ['Compra', 'Pagamento', 'Depósito', 'Retirada', 'Transferência']
for _ in range(num_transacoes):
    transacao_id = fake.unique.random_number(digits=10)
    cliente_id = random.choice(df_clientes['cliente_id'].tolist())
    data_transacao = fake.date_time_between(start_date='-1y', end_date='now')
    tipo_transacao = random.choice(tipos_transacao)
    valor = round(random.uniform(10, 1000), 2)
    descricao = fake.sentence(nb_words=5)
    transacoes_data.append({
        'transacao_id': transacao_id,
        'cliente_id': cliente_id,
        'data_transacao': data_transacao,
        'tipo_transacao': tipo_transacao,
        'valor': valor,
        'descricao': descricao
    })
df_transacoes = pd.DataFrame(transacoes_data)

# --- Geração do DataFrame de Empréstimos ---
emprestimos_data = []
tipos_emprestimo = ['Pessoal', 'Automotivo', 'Imobiliário', 'Consignado']
status_emprestimo = ['Aprovado', 'Pendente', 'Pago', 'Em Atraso', 'Negativado']
perfis_pagamento = ['Bom Pagador', 'Pagador Regular', 'Mau Pagador']
probabilidade_perfil = [0.6, 0.3, 0.1] # Probabilidades para cada perfil

for _ in range(num_emprestimos):
    emprestimo_id = fake.unique.random_number(digits=9)
    cliente_id = random.choice(df_clientes['cliente_id'].tolist())
    data_solicitacao = fake.date_time_between(start_date='-2y', end_date='-6mo')
    tipo_emprestimo = random.choice(tipos_emprestimo)
    valor_emprestimo = round(random.uniform(1000, 50000), 2)
    taxa_juros = round(random.uniform(0.01, 0.05), 4)
    prazo_meses = random.randint(12, 60)
    data_aprovacao = fake.date_time_between(start_date=data_solicitacao, end_date='-3mo') if random.random() < 0.8 else None
    status = random.choice(status_emprestimo)
    perfil = random.choices(perfis_pagamento, weights=probabilidade_perfil, k=1)[0]
    emprestimos_data.append({
        'emprestimo_id': emprestimo_id,
        'cliente_id': cliente_id,
        'data_solicitacao': data_solicitacao,
        'tipo_emprestimo': tipo_emprestimo,
        'valor_emprestimo': valor_emprestimo,
        'taxa_juros': taxa_juros,
        'prazo_meses': prazo_meses,
        'data_aprovacao': data_aprovacao,
        'status': status,
        'perfil_pagamento': perfil
    })
df_emprestimos = pd.DataFrame(emprestimos_data)

# --- Geração do DataFrame de Pagamentos (Simulação de Atrasos Contínuos Sem Perfil) ---
pagamentos_data = []
emprestimos_aprovados = df_emprestimos[df_emprestimos['data_aprovacao'].notna()]
data_atual_simulacao = datetime(2025, 4, 13) # Usando a data atual para referência

if not emprestimos_aprovados.empty:
    for index, emprestimo in emprestimos_aprovados.iterrows():
        prazo_meses = emprestimo['prazo_meses']
        valor_emprestimo = emprestimo['valor_emprestimo']
        data_aprovacao = emprestimo['data_aprovacao']
        valor_parcela_teorica = round(valor_emprestimo / prazo_meses, 2)

        # Simula o tipo de comportamento de atraso para este empréstimo
        tipo_atraso = random.choices(['nenhum', 'inicio', 'continuo_apos_pago', 'poucos'], weights=[0.6, 0.1, 0.2, 0.1], k=1)[0]
        primeiro_atraso_ocorreu = False
        meses_pago_inicialmente = random.randint(1, min(3, prazo_meses // 3)) if tipo_atraso == 'continuo_apos_pago' else 0
        num_atrasos_iniciais = random.randint(1, 3) if tipo_atraso == 'poucos' else 0
        atrasos_gerados = 0

        for i in range(1, prazo_meses + 1):
            data_vencimento = data_aprovacao + timedelta(days=i * 30)
            data_pagamento_simulada = None
            status_pagamento = 'Pendente'
            valor_pago = 0.0

            if data_vencimento < data_atual_simulacao:  # Parcela Vencida
                if tipo_atraso == 'inicio':
                    status_pagamento = 'Em Atraso'
                elif tipo_atraso == 'continuo_apos_pago':
                    if i <= meses_pago_inicialmente:
                        data_pagamento_simulada = data_vencimento - timedelta(days=random.randint(0, 3))
                        valor_pago = round(random.uniform(valor_parcela_teorica * 0.9, valor_parcela_teorica * 1.1), 2)
                        status_pagamento = 'Pago'
                    else:
                        status_pagamento = 'Em Atraso'
                elif tipo_atraso == 'poucos':
                    if atrasos_gerados < num_atrasos_iniciais and random.random() < 0.6:
                        status_pagamento = 'Em Atraso'
                        atrasos_gerados += 1
                    else:
                        data_pagamento_simulada = data_vencimento - timedelta(days=random.randint(0, 3))
                        valor_pago = round(random.uniform(valor_parcela_teorica * 0.9, valor_parcela_teorica * 1.1), 2)
                        status_pagamento = 'Pago'
                else: # tipo_atraso == 'nenhum' (maioria dos clientes)
                    data_pagamento_simulada = data_vencimento - timedelta(days=random.randint(0, 3)) if random.random() < 0.9 else None
                    valor_pago = round(random.uniform(valor_parcela_teorica * 0.9, valor_parcela_teorica * 1.1), 2) if data_pagamento_simulada else 0.0
                    status_pagamento = 'Pago' if data_pagamento_simulada else 'Em Atraso' if random.random() < 0.1 else 'Pago' # Pequena chance de atraso isolado

            pagamentos_data.append({
                'pagamento_id': fake.unique.random_number(digits=10),
                'emprestimo_id': emprestimo['emprestimo_id'],
                'numero_parcela': i,
                'data_vencimento': data_vencimento.date(),
                'data_pagamento': data_pagamento_simulada.date() if data_pagamento_simulada else None,
                'valor_pago': valor_pago if status_pagamento == 'Pago' else 0.0,
                'status_pagamento': status_pagamento
            })
df_pagamentos = pd.DataFrame(pagamentos_data)

# --- Função para escrever o dataframe em uma tabela PostgreSQL ---
def write_dataframe_to_rds(dataframe, table_name, db_host, db_port, db_name, db_user, db_password):
    try:
        engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
        dataframe.to_sql(table_name, engine, if_exists='append', index=False)
        print(f"Dataframe '{table_name}' escrito com sucesso no RDS.")
    except Exception as e:
        print(f"Erro ao escrever o dataframe '{table_name}' no RDS: {e}")
    finally:
        if 'engine' in locals():
            engine.dispose()

# --- Configurações do Banco de Dados RDS ---
DB_HOST = 'db-banco.c07gis2620tg.us-east-1.rds.amazonaws.com'
DB_PORT = '5432'
DB_NAME = 'db_banco'
DB_USER = 'postgres'
DB_PASSWORD = 'XqNuKeQw12345'

# --- Escrevendo os dataframes nas tabelas do RDS ---
write_dataframe_to_rds(df_clientes, 'clientes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
write_dataframe_to_rds(df_transacoes, 'transacoes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
write_dataframe_to_rds(df_emprestimos, 'emprestimos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
write_dataframe_to_rds(df_pagamentos, 'pagamentos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)

print("Geração e escrita dos dataframes concluídas.")

Novo registro adicionado à tabela 'clientes' no RDS.
Novo registro adicionado à tabela 'emprestimos' no RDS.
Novo registro adicionado à tabela 'pagamentos' no RDS.
Novo registro adicionado à tabela 'clientes' no RDS.


KeyboardInterrupt: 